In [7]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
# Mostrar DataFrames sin saltos de línea
pd.set_option('display.expand_frame_repr', False)
import ast



In [8]:
df = pd.read_csv("inference_results_multiple_cat.csv", sep=";", na_values="NaN")
df.drop(["inference_time","output"], axis=1,inplace=True)
df.head()

,img,category_llava
0,snev_NA_5603_53937889425__b.jpg,"ANIMALS, OTHER ABIOTIC FEATURES"
1,snev_NA_1507_53978219459__b.jpg,"HERITAGE AND CULTURE, TOWNS AND VILLAGES"
2,snev_NA_1605_53698444049__b.jpg,"HERITAGE AND CULTURE, KNOWLEDGE"
3,snev_NA_5603_53731370861__b.jpg,"LANDFORMS, OTHER ABIOTIC FEATURES"
4,snev_NA_1604_53702414354__b.jpg,"HERITAGE AND CULTURE, TOWNS AND VILLAGES"


In [10]:
df[["category_1","category_2"]] = df["category_llava"].str.split(',',n=2, expand=True).iloc[:, :2]
df["category_1"].str.strip()
df["category_2"].str.strip()
df.head()

,img,category_llava,category_1,category_2
0,snev_NA_5603_53937889425__b.jpg,"ANIMALS, OTHER ABIOTIC FEATURES",ANIMALS,OTHER ABIOTIC FEATURES
1,snev_NA_1507_53978219459__b.jpg,"HERITAGE AND CULTURE, TOWNS AND VILLAGES",HERITAGE AND CULTURE,TOWNS AND VILLAGES
2,snev_NA_1605_53698444049__b.jpg,"HERITAGE AND CULTURE, KNOWLEDGE",HERITAGE AND CULTURE,KNOWLEDGE
3,snev_NA_5603_53731370861__b.jpg,"LANDFORMS, OTHER ABIOTIC FEATURES",LANDFORMS,OTHER ABIOTIC FEATURES
4,snev_NA_1604_53702414354__b.jpg,"HERITAGE AND CULTURE, TOWNS AND VILLAGES",HERITAGE AND CULTURE,TOWNS AND VILLAGES


In [23]:
df.drop(["id"], axis=1,inplace=True, errors="ignore")
df.insert(0, "id",df.index+1)
df


,id,img,category_llava,category_1,category_2
0,1,snev_NA_5603_53937889425__b.jpg,"ANIMALS, OTHER ABIOTIC FEATURES",ANIMALS,OTHER ABIOTIC FEATURES
1,2,snev_NA_1507_53978219459__b.jpg,"HERITAGE AND CULTURE, TOWNS AND VILLAGES",HERITAGE AND CULTURE,TOWNS AND VILLAGES
2,3,snev_NA_1605_53698444049__b.jpg,"HERITAGE AND CULTURE, KNOWLEDGE",HERITAGE AND CULTURE,KNOWLEDGE
3,4,snev_NA_5603_53731370861__b.jpg,"LANDFORMS, OTHER ABIOTIC FEATURES",LANDFORMS,OTHER ABIOTIC FEATURES
4,5,snev_NA_1604_53702414354__b.jpg,"HERITAGE AND CULTURE, TOWNS AND VILLAGES",HERITAGE AND CULTURE,TOWNS AND VILLAGES
...,...,...,...,...,...
5061,5062,snev_NA_1507_53978219469__b.jpg,"TOWNS AND VILLAGES, HERITAGE AND CULTURE",TOWNS AND VILLAGES,HERITAGE AND CULTURE
5062,5063,snev_NA_1508_53801543815__b.jpg,"HERITAGE AND CULTURE, TOWNS AND VILLAGES",HERITAGE AND CULTURE,TOWNS AND VILLAGES
5063,5064,snev_NA_1410_53737712162__b.jpg,"TOWNS AND VILLAGES, HERITAGE AND CULTURE",TOWNS AND VILLAGES,HERITAGE AND CULTURE
5064,5065,snev_NA_1507_53801114326__b.jpg,"HERITAGE AND CULTURE, TOWNS AND VILLAGES",HERITAGE AND CULTURE,TOWNS AND VILLAGES


In [34]:
# Para cada carpeta con los clústers, leer el directorio y machear

# 1. Cargar df

base_dir = "clusters"
data = []

for folder in os.listdir(base_dir):
    for file in os.listdir(os.path.join(base_dir, folder)):
        data.append([int(folder), file]) 

df_clusters = pd.DataFrame(data, columns=["cluster", "img"])

# 2. Unir con el df original por imagen

df_clusters = df_clusters.merge(df, left_on="img", right_on="img", how="left")
df_clusters = df_clusters.sort_values(by="cluster", ascending=True)
df_clusters

df_clusters.drop(["id"], axis=1,inplace=True, errors="ignore")
df_clusters.insert(0, "id",df.index+1)
df_clusters

,id,cluster,img,category_llava,category_1,category_2
809,1,-1,snev_NA_1214_53947434048__b.jpg,"TRACKS AND TRAILS, VEGETATION AND HABITATS",TRACKS AND TRAILS,VEGETATION AND HABITATS
950,2,-1,snev_NA_1507_53702060056__b.jpg,"HERITAGE AND CULTURE, KNOWLEDGE, SPIRITUAL OR ...",HERITAGE AND CULTURE,KNOWLEDGE
949,3,-1,snev_NA_1413_53577666166__b.jpg,"HERITAGE AND CULTURE, KNOWLEDGE",HERITAGE AND CULTURE,KNOWLEDGE
948,4,-1,snev_NA_3474_53736356798__b.jpg,"HERITAGE AND CULTURE, TOWNS AND VILLAGES",HERITAGE AND CULTURE,TOWNS AND VILLAGES
947,5,-1,snev_NA_1507_53545142882__b.jpg,"HERITAGE AND CULTURE, KNOWLEDGE",HERITAGE AND CULTURE,KNOWLEDGE
...,...,...,...,...,...,...
2383,5062,261,snev_NA_1508_53701177832__b.jpg,"TOWNS AND VILLAGES, HERITAGE AND CULTURE",TOWNS AND VILLAGES,HERITAGE AND CULTURE
2382,5063,261,snev_NA_1508_53702385654__b.jpg,"TOWNS AND VILLAGES, HERITAGE AND CULTURE",TOWNS AND VILLAGES,HERITAGE AND CULTURE
2381,5064,261,snev_NA_1507_53517780834__b.jpg,"HERITAGE AND CULTURE, TOWNS AND VILLAGES",HERITAGE AND CULTURE,TOWNS AND VILLAGES
2388,5065,261,snev_NA_5374_53628085617__b.jpg,"HERITAGE AND CULTURE, TOWNS AND VILLAGES",HERITAGE AND CULTURE,TOWNS AND VILLAGES


In [35]:
df_clusters.to_csv("clusters_experiment_28.csv", sep=";", index=False)